# Import

In [1]:
from datasets import load_dataset
import pandas as pd
import torch
from torch.utils.data import Dataset
from underthesea import word_tokenize
from pyvi import ViTokenizer
import os
from datasets import load_dataset, load_from_disk
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import unidecode
import re
import random
from bs4 import BeautifulSoup
import contractions
import pandas as pd
import torch
from pathlib import Path
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from torchtext.data.metrics import bleu_score
from torchmetrics import Recall, Precision, FBetaScore, Accuracy
import numpy as np

/home/phamngocthi/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/phamngocthi/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# config

In [2]:
config = {}
# Dataset
config["lang_src"] = "noise_vi"
config["lang_tgt"] = "vi"
config["ratio"] = 0.3
config["sound_char_ratio"] = 0.3
config["no_accent_char_ratio"] = 0.2 
config["remove_word_ratio"] = 0.1
config["multi_word_ratio"] = 0.1
config["face_char_ratio"] = 0.3
config["num_noise"] = 3
config["train_dataset_path"] = "./dataset/9_dataset/train.csv"

# Device
config["device"] = "cuda"

# Model
config["d_model"] = 512
config["num_encoder"] = 6
config["num_decoder"] = 6
config["nhead"] = 8
config["d_ff"] = 2048
config["dropout"] = 0.1
config["max_len"] = 100

# Word tokenizer
config["underthesea"] = True
config["pyvi"] = False

# Optimier: Adam
config["weight_decay"] = 0
config["lr"] = 1e-4
config["lr_scheduler"] = False
config["eps"] = 1e-9
config["betas"] = (0.9, 0.98)

# Scheduler (config["lr_scheduler"] = True)
## LambdaLR
config["lambdalr"] = False
config["warmup_steps"] = 4000
## StepLR
config["steplr"] = False
config["step_size_steplr"] = 1
config["gamma_steplr"] = 0.5

# Loss function: Cross_entropy
config["label_smoothing"] = 0.1

# Train
config["batch_size_train"] = 32
config["batch_size_validation"] = 32
config["num_epochs"] = 5
config["train_size"] = 0.9
config["num_bleu_validation"] = 100

# Validation Bleu metric
config["max_beam"] = 1

# Test
config["beam_test"] = 2
config["data_test"] = "vietnamese_spelling_correction/dataset/dataset.csv"
config["f_beta"] = 0.5

# Save
config["tokenizer_file"] = "./dataset/9_dataset/tokenizer_{}.json"
config["experiment_name"] = "results_train"
config["save_config"] = "config/config_{0}.json"
config["model_folder"] = "weights"
config["model_basename"] = "tmodel_"
config["data_path"] = "data"
config["save_config_pattern"] = "config/config_*"

# Different
config["preload"] = "latest"

In [3]:
# load the dataset
dataset = load_dataset("mt_eng_vietnamese", "iwslt2015-en-vi")
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 133318
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1269
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1269
    })
})

In [4]:
# convert the dataset to a pandas dataframe
df = pd.DataFrame(dataset['train'])
df.head()

,translation
0,{'en': 'Rachel Pike : The science behind a cli...
1,"{'en': 'In 4 minutes , atmospheric chemist Rac..."
2,{'en': 'I &apos;d like to talk to you today ab...
3,{'en': 'Headlines that look like this when the...
4,{'en': 'They are both two branches of the same...


In [5]:
# convert ["translation"] = {"en": "source", "vi": "target"} => {en: "source", vi: "target"}
df[config["lang_src"]] = df["translation"].apply(lambda x: x["en"])
df[config["lang_tgt"]] = df["translation"].apply(lambda x: x["vi"])
df = df.drop(columns=["translation"])
df.head()

,noise_vi,vi
0,Rachel Pike : The science behind a climate hea...,Khoa học đằng sau một tiêu đề về khí hậu
1,"In 4 minutes , atmospheric chemist Rachel Pike...","Trong 4 phút , chuyên gia hoá học khí quyển Ra..."
2,I &apos;d like to talk to you today about the ...,Tôi muốn cho các bạn biết về sự to lớn của nhữ...
3,Headlines that look like this when they have t...,Có những dòng trông như thế này khi bàn về biế...
4,They are both two branches of the same field o...,Cả hai đều là một nhánh của cùng một lĩnh vực ...


# x5 the dataset

In [6]:
df_keep = df
df = pd.concat([df]*2)
df = df.reset_index(drop=True)
df_keep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133318 entries, 0 to 133317
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   noise_vi  133318 non-null  object
 1   vi        133318 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


# create noise function

In [7]:
# create noise
# Lỗi tương đồng về hình dạng của chữ
chars_regrex = '[aàảãáạăằẳẵắặâầẩẫấậoòỏõóọôồổỗốộơờởỡớợeèẻẽéẹêềểễếệuùủũúụưừửữứựiìỉĩíịyỳỷỹýỵnvm]'
same_chars = {
    'a': ['á', 'à', 'ả', 'ã', 'ạ', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ'],
    'o': ['ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ','ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'q'],
    'e': ['é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ê'],
    'u': ['ú', 'ù', 'ủ', 'ũ', 'ụ', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'ư'],
    'i': ['í', 'ì', 'ỉ', 'ĩ', 'ị'],
    'y': ['ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'v'],
    'n': ['m'],
    'v': ['y'],
    'm': ['n'],
}

def _char_regrex(text):
    match_chars = re.findall(chars_regrex, text)
    return match_chars

def _random_replace(text, match_chars):
    replace_char = match_chars[np.random.randint(low=0, high=len(match_chars), size=1)[0]]
    insert_chars = same_chars[unidecode.unidecode(replace_char)]
    insert_char = insert_chars[np.random.randint(low=0, high=len(insert_chars), size=1)[0]]
    text = text.replace(replace_char, insert_char, 1)

    return text

def replace_face_char(text):
    match_chars = _char_regrex(text)
    if len(match_chars) == 0:
        return text
    text = _random_replace(text, match_chars)
    return text
# End: lỗi tương đồng về hình dạng của chữ

# Lỗi tương đồng về âm thanh khi đọc
three_char = {
    "ngh": ["ng"]
}
two_char = {
    "ng": ["g", "n"],
    "gi": ["d", "v"],
    "gh": ["g"],
    "ch": ["tr", "c"],
    "tr": ["ch"],
    "ph": ["p"],
    "qu": ["q"],
    "kh": ["k"],
    "iê": ["i"],
    "iế": ["í"],
    "iệ": ["ị"],
    "iề": ["ì"],
    "iể": ["ỉ"],
    "iễ": ["ĩ"],
}
one_char = {
    "g": ["gh", "r"],
    "r": ["gh", "g"],
    "d": ["gi", "v"],
    "k": ["kh"],
    "ph": ["p"],
    "p": ["ph"],
    "q": ["qu"],
    "s": ["x"],
    "x": ["s"],
    "l": ["n"],
    "n": ["l"],
    "v": ["gi", "d"],
    "y": ["i"],
    "ý": ["í"],
    "ỳ": ["ì"],
    "ỷ": ["ỉ"],
    "ỹ": ["ĩ"],
    "ỵ": ["ị"],
    "i": ["y"],
    "í": ["ý"],
    "ì": ["ỳ"],
    "ỉ": ["ỷ"],
    "ĩ": ["ỹ"],
    "ị": ["ỵ"],
}

def replace_sound_char(word):
    n_chars = [three_char, two_char, one_char]
    for n_char in n_chars:
        for char in n_char:
            if char in word:
                return word.replace(char, random.choice(n_char[char]))
    return word
# End: lỗi tương đồng về mặt âm thanh khi đọc

# Lỗi gõ không dấu
def replace_no_accent_char(word):
    return unidecode.unidecode(word)
# End: lỗi gõ không dấu

# Lỗi gõ lặp lại chữ
def multi_word(word):
    return f"{word} {word}"
# End: lỗi gõ lặp lại chữ

# Lỗi mất chữ
def remove_word(word):
    return ""
# End: lỗi mất chữ

# create noise function
def random_ratio(array: list[int]=[], ratios: float=0.9, size: int=0):
    # print(f"{ratios = }")
    if len(array) != len(ratios):
        raise ValueError("The length of array and ratios must be the same")
    result_array = [-1] * len(array)
    for i in range(len(array)):
        num_element = int(size * ratios[i])
        sub_array = [array[i]] * num_element
        result_array += sub_array
    # print(f"{result_array = }")
    for i in range(len(result_array)):
        if result_array[i] == -1:
            result_array[i] = array[-1]
    np.random.shuffle(result_array)
    return result_array

def create_noise_random_choice(vi_sent):
    words = vi_sent.split()
    funcs_noise = [replace_face_char, replace_sound_char, replace_no_accent_char, remove_word, multi_word]
    # funcs_noise = [replace_face_char, replace_sound_char, replace_no_accent_char, multi_word]
    if len(words) <= 1:
        return vi_sent
    func_noise = funcs_noise[random.randint(0, len(funcs_noise) - 1)]
    idx = random.randint(0, len(words) - 1)
    words[idx] = func_noise(words[idx])
    res = ""
    for i in range(len(words)):
        if words[i] == "":
            continue
        res = res + " " + words[i]
    return res.strip()

def create_noise(config, vi_sent):
    ratio = config["ratio"]
    face_char_ratio = round(config["face_char_ratio"] * ratio, 3)
    sound_char_ratio = round(config["sound_char_ratio"] * ratio, 3)
    no_accent_char_ratio = round(config["no_accent_char_ratio"] * ratio, 3)
    remove_word_ratio = round(config["remove_word_ratio"] * ratio, 3)
    multi_word_ratio = round(config["multi_word_ratio"] * ratio, 3)
    words = vi_sent.split()
    funcs_noise = [replace_face_char, replace_sound_char, replace_no_accent_char, remove_word, multi_word]
    # funcs_noise = [replace_face_char, replace_sound_char, replace_no_accent_char, multi_word]
    ratios = [face_char_ratio,
              sound_char_ratio,
              no_accent_char_ratio,
              remove_word_ratio,
              multi_word_ratio,
              1 - (face_char_ratio + sound_char_ratio + no_accent_char_ratio + remove_word_ratio + multi_word_ratio)]
            #   1 - (face_char_ratio + sound_char_ratio + no_accent_char_ratio + multi_word_ratio)]
    rand_error = random_ratio(
        array=list(range(len(ratios))),
        ratios=ratios,
        size = len(words)
    )
    # print(f"{ rand_error = }")
    res = ""
    for i in range(len(words)):
        if rand_error[i] == len(ratios) - 1:
            res = res + " " + words[i]
        else:
            # print(f"{rand_error[i] = }")
            change_word = (funcs_noise[rand_error[i]])(words[i])
            if change_word != "":
                res = res + " " + change_word
    return res.strip()
# end create noise

def clean_data(text, lang):
    text = BeautifulSoup(text, "html.parser").get_text()
    text = (text.lower()).replace(" '", "'")
    if lang == "en":
        text = contractions.fix(text)
    return handle_special_char(text)

def handle_special_char(sent):
    sent = re.sub(r'([.,!?;(){}\[\]])', r' \1 ', sent)
    sent = re.sub(r'\s{2,}', ' ', sent)
    sent = sent.strip()
    return sent

# add noise in dataset

In [8]:
# assign source by target
df[config["lang_src"]] = df[config["lang_tgt"]]
df_keep[config["lang_src"]] = df_keep[config["lang_tgt"]]
df_keep.head()
df.head()

,noise_vi,vi
0,Khoa học đằng sau một tiêu đề về khí hậu,Khoa học đằng sau một tiêu đề về khí hậu
1,"Trong 4 phút , chuyên gia hoá học khí quyển Ra...","Trong 4 phút , chuyên gia hoá học khí quyển Ra..."
2,Tôi muốn cho các bạn biết về sự to lớn của nhữ...,Tôi muốn cho các bạn biết về sự to lớn của nhữ...
3,Có những dòng trông như thế này khi bàn về biế...,Có những dòng trông như thế này khi bàn về biế...
4,Cả hai đều là một nhánh của cùng một lĩnh vực ...,Cả hai đều là một nhánh của cùng một lĩnh vực ...


In [9]:
# create noise for source
df[config["lang_tgt"]] = df[config["lang_tgt"]].apply(lambda x: clean_data(x, config["lang_tgt"]))
df[config["lang_src"]] = df[config["lang_src"]].apply(lambda x: create_noise(config, clean_data(x, config["lang_src"])))
df_keep[config["lang_tgt"]] = df_keep[config["lang_tgt"]].apply(lambda x: clean_data(x, config["lang_tgt"]))
df_keep[config["lang_src"]] = df_keep[config["lang_src"]].apply(lambda x: clean_data(x, config["lang_src"]))
df.head()

/tmp/ipykernel_7883/2741826605.py:176: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


,noise_vi,vi
0,khoa học đằng sau một tiêu đề về khí hậu,khoa học đằng sau một tiêu đề về khí hậu
1,"trong 4 phút phút , chuyên gia gia hoá học khí...","trong 4 phút , chuyên gia hoá học khí quyển ra..."
2,tôy muốn cho các bạn biết giề sụ to lớn của nh...,tôi muốn cho các bạn biết về sự to lớn của nhữ...
3,có nhung dòng trông như thế này khi bàn vể biế...,có những dòng trông như thế này khi bàn về biế...
4,cả hai đều là một nhánh của cùg một lĩnh vực t...,cả hai đều là một nhánh của cùng một lĩnh vực ...


In [10]:
# merge df and df_keep
df = pd.concat([df, df_keep])
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399954 entries, 0 to 399953
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   noise_vi  399954 non-null  object
 1   vi        399954 non-null  object
dtypes: object(2)
memory usage: 6.1+ MB


In [11]:
# filter especial char not in vietnamese
def filter_especial_char(sent):
    especial_char = "wzjzf"
    for char in sent:
        if char in especial_char:
            return False
    return True

# filter df with src and tgt
df = df[df[config["lang_src"]].apply(lambda x: filter_especial_char(x))]
df = df[df[config["lang_tgt"]].apply(lambda x: filter_especial_char(x))]
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370680 entries, 0 to 370679
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   noise_vi  370680 non-null  object
 1   vi        370680 non-null  object
dtypes: object(2)
memory usage: 5.7+ MB


# create tokenizer

In [12]:
def get_all_sentences(dataset, lang):
    for item in dataset:
        yield item

def get_or_build_tokenizer(config, dataset, lang):
    tokenizer_path = Path(config['tokenizer_file'].format(lang))
    if not Path.exists(tokenizer_path):
        tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(vocab_size=50000, special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)
        tokenizer.train_from_iterator(get_all_sentences(dataset, lang), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    return tokenizer

def get_tokenizer(config, dataset):
    tokenizer_src = get_or_build_tokenizer(config, dataset[config["lang_src"]], config["lang_src"])
    tokenizer_tgt = get_or_build_tokenizer(config, dataset[config["lang_tgt"]], config["lang_tgt"])
    return tokenizer_src, tokenizer_tgt

In [13]:
tokenizer_src, tokenizer_tgt = get_tokenizer(config, df)

In [14]:
tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()

(27601, 14872)

# filter len tokenzier on df

In [15]:
def create_condition(pair, config):
    sent_src = pair[config["lang_src"]]
    sent_tgt = pair[config["lang_tgt"]]
    if type(sent_src) != str or type(sent_tgt) != str:
        return False
    
    token_ids_src = tokenizer_src.encode(sent_src).ids
    token_ids_tgt = tokenizer_tgt.encode(sent_tgt).ids

    min_len = min(len(token_ids_src), len(token_ids_tgt))
    max_len = max(len(token_ids_src), len(token_ids_tgt))
    if min_len > 10 and max_len <= config["max_len"] - 4:
        return True
    return False

df = df[df.apply(lambda x: create_condition(x, config), axis=1)].reset_index(drop=True)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303371 entries, 0 to 303370
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   noise_vi  303371 non-null  object
 1   vi        303371 non-null  object
dtypes: object(2)
memory usage: 4.6+ MB


,noise_vi,vi
0,"trong 4 phút phút , chuyên gia gia hoá học khí...","trong 4 phút , chuyên gia hoá học khí quyển ra..."
1,tôy muốn cho các bạn biết giề sụ to lớn của nh...,tôi muốn cho các bạn biết về sự to lớn của nhữ...
2,có nhung dòng trông như thế này khi bàn vể biế...,có những dòng trông như thế này khi bàn về biế...
3,cả hai đều là một nhánh của cùg một lĩnh vực t...,cả hai đều là một nhánh của cùng một lĩnh vực ...
4,các tiêu tiêu đề đâi trông như thế nàỷ khi ban...,các tiêu đề gần đây trông như thế này khi ban ...
...,...,...
303366,mối liên kết chung trong những việc chúng tôi ...,mối liên kết chung trong những việc chúng tôi ...
303367,"nelson maldela từng nói giữa những năm 2000 , ...","nelson maldela từng nói giữa những năm 2000 , ..."
303368,nó là do con người và có thể ngăn chặn và diệt...,nó là do con người và có thể ngăn chặn và diệt...
303369,tôi muốn kết luận rằng hành động của hàng ngàn...,tôi muốn kết luận rằng hành động của hàng ngàn...


In [16]:
# delete tokenizer file
def delete_tokenizer_file(config):
    tokenizer_file = Path(config["tokenizer_file"].format(config["lang_src"]))
    if Path.exists(tokenizer_file):
        Path.unlink(tokenizer_file)
    tokenizer_file = Path(config["tokenizer_file"].format(config["lang_tgt"]))
    if Path.exists(tokenizer_file):
        Path.unlink(tokenizer_file)

delete_tokenizer_file(config)

In [17]:
tokenizer_src, tokenizer_tgt = get_tokenizer(config, df)

In [18]:
tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()

(26666, 14128)

In [19]:
df

,noise_vi,vi
0,"trong 4 phút phút , chuyên gia gia hoá học khí...","trong 4 phút , chuyên gia hoá học khí quyển ra..."
1,tôy muốn cho các bạn biết giề sụ to lớn của nh...,tôi muốn cho các bạn biết về sự to lớn của nhữ...
2,có nhung dòng trông như thế này khi bàn vể biế...,có những dòng trông như thế này khi bàn về biế...
3,cả hai đều là một nhánh của cùg một lĩnh vực t...,cả hai đều là một nhánh của cùng một lĩnh vực ...
4,các tiêu tiêu đề đâi trông như thế nàỷ khi ban...,các tiêu đề gần đây trông như thế này khi ban ...
...,...,...
303366,mối liên kết chung trong những việc chúng tôi ...,mối liên kết chung trong những việc chúng tôi ...
303367,"nelson maldela từng nói giữa những năm 2000 , ...","nelson maldela từng nói giữa những năm 2000 , ..."
303368,nó là do con người và có thể ngăn chặn và diệt...,nó là do con người và có thể ngăn chặn và diệt...
303369,tôi muốn kết luận rằng hành động của hàng ngàn...,tôi muốn kết luận rằng hành động của hàng ngàn...


# save dataset

In [20]:
# shuffle df
df = df.sample(frac=1).reset_index(drop=True)
df.info()
# save dataset
df.to_csv(config["train_dataset_path"], index=False)
# load dataset
df = pd.read_csv(config["train_dataset_path"])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303371 entries, 0 to 303370
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   noise_vi  303371 non-null  object
 1   vi        303371 non-null  object
dtypes: object(2)
memory usage: 4.6+ MB


In [21]:
# get max len token id of noise_vi and vi
def get_max_len_token_id(dataset, lang):
    max_len = 0
    for item in dataset:
        if (type(item) != str):
            continue
        if lang == config["lang_src"]:
            token_len = len(tokenizer_src.encode(item).ids)
            max_len = max(max_len, token_len)
        else:
            token_len = len(tokenizer_tgt.encode(item).ids)
            max_len = max(max_len, token_len)
    return max_len

max_len_src = get_max_len_token_id(df[config["lang_src"]], config["lang_src"])
max_len_tgt = get_max_len_token_id(df[config["lang_tgt"]], config["lang_tgt"])

max_len_src, max_len_tgt

(96, 96)

In [22]:
# count number source != target
cnt_diff, cnt_same = 0, 0
for id in range(len(df)):
    src = df.iloc[id][config["lang_src"]]
    tgt = df.iloc[id][config["lang_tgt"]]
    cnt_diff += src != tgt
    cnt_same += src == tgt

cnt_diff, cnt_same, cnt_diff + cnt_same, len(df)

(184197, 119174, 303371, 303371)

# split train and val

In [24]:
# read custom_dataset.csv
df = pd.read_csv(config["train_dataset_path"])

In [25]:
# split 90% train, 10% validation
train_size = int(len(df) * config["train_size"])
# shuffle the dataset
train_df = df[:train_size]
validation_df = df[train_size:]

In [26]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273033 entries, 0 to 273032
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   noise_vi  273033 non-null  object
 1   vi        273033 non-null  object
dtypes: object(2)
memory usage: 4.2+ MB


In [27]:
validation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30338 entries, 273033 to 303370
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   noise_vi  30338 non-null  object
 1   vi        30338 non-null  object
dtypes: object(2)
memory usage: 474.2+ KB


In [28]:
# create test bleu dataset 100 samples from validation dataset
test_validation_df = validation_df[:config["num_bleu_validation"]]
test_validation_df.info()
# create test bleu train dataset 100 samples from train dataset
test_train_df = train_df[:config["num_bleu_validation"]]
test_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 273033 to 273132
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   noise_vi  100 non-null    object
 1   vi        100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   noise_vi  100 non-null    object
 1   vi        100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


# custom dataset

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, ds, src_lang, tgt_lang):
        super().__init__()
        self.ds = ds
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        src_target_pair = self.ds.iloc[idx]
        src_text = src_target_pair[self.src_lang]
        tgt_text = src_target_pair[self.tgt_lang]

        return (src_text, tgt_text)

In [ ]:
custom_train_df = CustomDataset(train_df, config["lang_src"], config["lang_tgt"])
custom_validation_df = CustomDataset(validation_df, config["lang_src"], config["lang_tgt"])
custom_bleu_validation_df = CustomDataset(test_validation_df, config["lang_src"], config["lang_tgt"])
custom_bleu_train_df = CustomDataset(test_train_df, config["lang_src"], config["lang_tgt"])

In [ ]:
custom_train_df.__getitem__(0)

In [ ]:
custom_validation_df.__getitem__(0)

In [ ]:
custom_bleu_validation_df.__getitem__(0)

In [ ]:
custom_bleu_train_df.__getitem__(0)